In [1]:
import numpy as np
import cv2 as cv
import os
import re
from Visualization import *
from hough_transform import *

import matplotlib.pyplot as plt

Visualization.py loaded


In [2]:
video_dir = os.path.join('Lane_Parameters', 'Lane_Videos')
lane_dir = os.path.join('Lane_Parameters', 'Lane_Parameters')
def get_video(video_num):
    return os.path.join(video_dir, 'IMG_'+ video_num + '.MOV')


def get_lanes(video_num, frame_num, factor = 1):
    
    p = re.compile('[0-9]*,[0-9]*')
    file = open(os.path.join(lane_dir, video_num, str(frame_num) + '.txt'))
    lanes = []

    for line in file:
        points = p.findall(line)
        for i, point in enumerate(points):
            points[i] = tuple(map(int, point.split(',')))
            points[i] = int(points[i][0]/factor), int(points[i][1]/factor)
        
        lanes.append(points)
    file.close()
    return lanes

In [ ]:
video_num = '0250'
vid = cv.VideoCapture(get_video(video_num))

frame_num = 1000

for i in range(frame_num):
    sucess, orignal_frame = vid.read()
    if not sucess:
        break
vid.release()


img = orignal_frame.copy()

factor = 2

img = cv.resize(img,(0,0), fx=1/factor, fy=1/factor)

PLTdraw(orignal_frame)

In [ ]:
def draw_dots(img, dots, radius=2, color=(255,0,0), thickness=2):
    for dot in dots:
        img = cv.circle(img, dot, 2, color, thickness)
    return img

def draw_lanes(img_org, lanes, radius=3, colorLst=None, thickness=1):
    img = img_org.copy()
    for lane in lanes:
        color = None
        if colorLst is None:
            color = random_color()
        img = draw_dots(img, lane, radius, color, thickness)
    return img

In [ ]:
import random as rand

def random_color():
    rand.seed(10)
    return rand.randint(0,255), rand.randint(0,255), rand.randint(0,255)

In [ ]:
lanes = get_lanes(video_num, frame_num, factor)
doted = draw_lanes(img, lanes)
PLTdraw(doted)

In [ ]:
def bgr2gray(bgr):
    return (bgr[:,:,2]*0.2126 + bgr[:,:,1]*0.7152 + bgr[:,:,0]*0.0722)

grayed = bgr2gray(img)
plt.imshow(grayed)

## Sobel Edge Detector 
By Tharm

In [ ]:
def sobel(grayed_frame, thresh=100):
    
    #Implementing image smoothing
    rows = grayed_frame.shape[0]
    cols = grayed_frame.shape[1]
    
    kernel = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])/9.0
    mag_smooth = np.zeros((rows,cols))
    
    #pad with zeros on the border for full blurring of image
    padded_gray = np.zeros((rows + 2, cols + 2))
    padded_gray[1:-1, 1:-1] = grayed_frame
    
    for x in range(rows):
        for y in range(cols):
            mag_smooth[x][y] = (kernel * padded_gray[x:x+3, y:y+3]).sum()
    
    ##Implementing sobel edge detector
    Gx = [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]
    Gy = [[-1, -2, -1], [0, 0, 0], [1, 2, 1]]
    
    mag_G = np.zeros((rows,cols))
    
    for i in range(1, rows - 1):
        for j in range(1, cols - 1):
            sumx = (Gx * mag_smooth[i-1:i+2, j-1:j+2]).sum()
            sumy = (Gy * mag_smooth[i-1:i+2, j-1:j+2]).sum()
            mag_G[i][j] = np.sqrt(sumx**2 + sumy**2)
            
    threshed = (mag_G > thresh).astype(np.uint8)
    
    binary_lut = np.zeros(256).astype("uint8")
    binary_lut[1] = 255
    
    threshed = cv.LUT(threshed.astype(np.uint8), binary_lut)
    
    return threshed

In [ ]:
def region_of_interest(img, vertices=None):
    # Define a blank matrix that matches the image height/width
    mask = np.zeros_like(img)
    
    img = img.astype('unit8')
    
    # Retrieve the number of color channels of the image
    # Will be 3 in this case because image is RGB
    if(len(img.shape)<3):
        channel_count = 1
    else:
        channel_count = img.shape[2]
    
    h, w = img.shape[0], img.shape[1]
    
    if vertices is None:
        vertices = np.array([
            (0, h),
            (w/2, h/2),
            (w, h),
        ], np.int8).astype('uint8')
    
    # Create a match color with the same color channel counts for fillPoly to know how to fill the mask
    match_mask_color = (255,) * channel_count
      
    # Fills the area bounded by the polygon with vertices in vertices
    cv.fillPoly(mask, vertices, match_mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv.bitwise_and(img, mask)
    
    return masked_image

In [ ]:
edged = sobel(grayed)


In [ ]:
edged = edged.astype('uint8')

In [ ]:
# h, w = edged.shape
# region_of_interest_vertices = np.array([
#     (0, h),
#     (w/2, h/2),
#     (w, h),
# ])
# region_of_interest_vertices = region_of_interest_vertices.astype('uint8')

cropped_image = region_of_interest(edged)

plt.imshow(cropped_image)

In [ ]:
accumulator, point_stack = Hough_lines(cropped_image, 70)

In [ ]:
points_coord = [] #To store the coordinates of the lines
# point_stack = point_stack[1:]
overlay = img.copy()
for i in point_stack:
    points = hough_intersect(i[0], i[1], img)
    if(len(points)!=2):
        continue
    points_coord.append([list(point) for point in points])
    cv.line(overlay, points[0], points[1], (0, 0, 255))
PLTdraw(overlay)

In [ ]:
points_coord

In [ ]:
?plt.hist

In [ ]:
def increase_contrast(img):
    [0, 255, 255, ...]
                
    return img